- Implement a complete RAG pipeline with a Stuff Documents chain.  
  Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요

- You must implement the chain manually.  
  체인을 수동으로 구현해야 합니다.

- Give a `ConversationBufferMemory` to the chain.  
  체인에 ConversationBufferMemory를 부여합니다.

- Use this document to perform RAG: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223

- Ask the following questions to the chain:
  - Is Aaronson guilty?
  - What message did he write in the table?
  - Who is Julia?


In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.2,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationBufferMemory(
    return_messages=True,
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [30]:
cache_dir = LocalFileStore("./.cache/")

loader = UnstructuredFileLoader("./files/document.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

In [5]:
print(docs[0])
len(docs)

page_content='Chapter 3\n\'There are three stages in your reintegration,\' said O\'Brien. \'There is learning, there is understanding, and there is acceptance. It is time for you to enter upon the second stage.\'\nAs always, Winston was lying flat on his back. But of late his bonds were looser. They still held him to the bed, but he could move his knees a little and could turn his head from side to side and raise his arms from the elbow. The dial, also, had grown to be less of a terror. He could evade its pangs if he was quick-witted enough: it was chiefly when he showed stupidity that O\'Brien pulled the lever. Sometimes they got through a whole session without use of the dial. He could not remember how many sessions there had been. The whole process seemed to stretch out over a long, indefinite time--weeks, possibly--and the intervals between the sessions might sometimes have been days, sometimes only an hour or two.\n\'As you lie there,\' said O\'Brien, \'you have often wondered--yo

34

In [35]:
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | RunnablePassthrough.assign(history=load_memory)
    | prompt
    | llm
)


def invoke_chain(question):
    result = chain.invoke(question)
    # print(result)

    memory.save_context(
        {"input": question},
        {"output": result.content},
    )

In [37]:
invoke_chain("Aaronson 은 유죄인가요?")
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")
invoke_chain("Julia 는 누구인가요?")

모르겠습니다.그는 "자유는 노예다"라는 메시지를 썼고, 그 아래에 "둘과 둘은 다섯이다"라고 썼습니다.Julia는 Winston의 사랑하는 사람입니다.

In [38]:
load_memory({})

[HumanMessage(content='Aaronson 은 유죄인가요?'),
 AIMessage(content='모르겠습니다.'),
 HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'),
 AIMessage(content='그는 "자유는 노예다"라는 메시지를 썼고, 그 아래에 "둘과 둘은 다섯이다"라고 썼습니다.'),
 HumanMessage(content='Julia 는 누구인가요?'),
 AIMessage(content='Julia는 Winston의 사랑하는 사람입니다.')]

In [39]:
invoke_chain("Is Aaronson guilty?")
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

I don't know.He wrote "FREEDOM IS SLAVERY" and beneath it "TWO AND TWO MAKE FIVE."Julia is Winston's love.

In [40]:
# 이전 과제 SystemMessage 가 같이 출력되는 이유...? -> Streaming 때문이었음